In [5]:
import sqlite3

conn = sqlite3.connect('news.db')
curr = conn.cursor()
data = curr.execute('SELECT REPLACE(REPLACE(c1.content,"<p>",""),"</p>","") as parent_comment,REPLACE(REPLACE(c2.content,"<p>","") ,"</p>","")as child_comment FROM comment as c1, comment as c2 WHERE c1.id = c2.parent_id;')
dataTable = data.fetchall()
conn.close()

In [12]:
import sqlite3
from bs4 import BeautifulSoup

# Connect to the SQLite database
conn = sqlite3.connect('news.db')
cursor = conn.cursor()

# Query the database for comments and replies
cursor.execute('SELECT REPLACE(REPLACE(c1.content,"<p>",""),"</p>","") as parent_comment,REPLACE(REPLACE(c2.content,"<p>","") ,"</p>","")as child_comment FROM comment as c1, comment as c2 WHERE c1.id = c2.parent_id;')
data = cursor.fetchall()

# Close the database connection
conn.close()

# Clean up text by removing HTML elements
def clean_text(text):
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text(separator=' ', strip=True)
    return cleaned_text

# Example of cleaning comments and replies
cleaned_data = [(clean_text(comment), clean_text(reply)) for comment, reply in data]

/tmp/ipykernel_9743/4039238407.py:17: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [13]:
cleaned_data

[('They wanted it concealed because the chinese were paying them to let it happen',
  'Or blackmailing Biden'),
 ('Or blackmailing Biden', 'Free ice cream.'),
 ('Or blackmailing Biden',
  'I believe that Ukraine and China are blackmailing the Bidens and all the other corrupt politicians who have been on the take all these years.'),
 ('They wanted it concealed because the chinese were paying them to let it happen',
  'Joe Biden is bought, paid for, and owned by the Chairman Xi.'),
 ('Joe Biden is bought, paid for, and owned by the Chairman Xi.',
  "The world sees Joe Biden and Chairman Xi's boy ."),
 ('Joe Biden is bought, paid for, and owned by the Chairman Xi.',
  'Joe Biden America\'s "Manchurian Candidate" if there ever was ! !'),
 ('They wanted it concealed because the chinese were paying them to let it happen',
  'Or he’s afraid to challenge them.'),
 ('They wanted it concealed because the chinese were paying them to let it happen',
  "22,000+ Chinese illegals have crossed the sou

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.legacy.data import Field, BucketIterator, TabularDataset

import sqlite3
import spacy
from bs4 import BeautifulSoup

# Load English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

# Define the fields for comments and replies
comment_field = Field(sequential=True, tokenize=lambda x: [tok.text for tok in nlp(x)], init_token='<sos>', eos_token='<eos>', lower=True)
reply_field = Field(sequential=True, tokenize=lambda x: [tok.text for tok in nlp(x)], init_token='<sos>', eos_token='<eos>', lower=True)

# Define the fields mapping to your database columns
fields = [('comment', comment_field), ('reply', reply_field)]

# Connect to the SQLite database and execute the query
conn = sqlite3.connect('news.db')
cursor = conn.cursor()
cursor.execute('SELECT c1.content, c2.content FROM comment as c1, comment as c2 WHERE c1.id = c2.parent_id')
data = cursor.fetchall()
conn.close()

# Clean up text by removing HTML elements
def clean_text(text):
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text(separator=' ', strip=True)
    return cleaned_text

# Clean the data and create a TabularDataset
cleaned_data = [(clean_text(comment), clean_text(reply)) for comment, reply in data]
examples = [torchtext.legacy.data.Example.fromlist([comment, reply], fields) for comment, reply in cleaned_data]
dataset = torchtext.legacy.data.Dataset(examples, fields)

# Split the dataset into training and testing sets
train_data, test_data = dataset.split(split_ratio=0.8)

# Build the vocabulary
comment_field.build_vocab(train_data, min_freq=2)
reply_field.build_vocab(train_data, min_freq=2)

# Define the Seq2Seq model
class Seq2SeqModel(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, output_dim):
        super(Seq2SeqModel, self).__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, src):
        embedded = self.embedding(src)
        output, hidden = self.rnn(embedded)
        return self.fc(hidden.squeeze(0))

# Set hyperparameters
input_dim = len(comment_field.vocab)
output_dim = len(reply_field.vocab)
emb_dim = 256
hidden_dim = 512
batch_size = 64

# Create model and optimizer
model = Seq2SeqModel(input_dim, emb_dim, hidden_dim, output_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Create iterators
train_iterator, test_iterator = BucketIterator.splits(
    (train_data, test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.comment),
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    for batch in train_iterator:
        optimizer.zero_grad()
        output = model(batch.comment)
        loss = criterion(output, batch.reply)
        loss.backward()
        optimizer.step()

# Save the trained model
torch.save(model.state_dict(), 'seq2seq_model.pth')



ModuleNotFoundError: No module named 'torchtext.legacy'

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchtext.utils import download_from_url, extract_archive

import sqlite3
import spacy
from bs4 import BeautifulSoup

# Load English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

# Connect to the SQLite database and execute the query
conn = sqlite3.connect('news.db')
cursor = conn.cursor()
cursor.execute('SELECT c1.content, c2.content FROM comment as c1, comment as c2 WHERE c1.id = c2.parent_id')
data = cursor.fetchall()
conn.close()

# Clean up text by removing HTML elements
def clean_text(text):
    soup = BeautifulSoup(text, 'html.parser')
    cleaned_text = soup.get_text(separator=' ', strip=True)
    return cleaned_text

# Clean the data
cleaned_data = [(clean_text(comment), clean_text(reply)) for comment, reply in data]

# Tokenize text using spaCy
def tokenize(text):
    return [tok.text for tok in nlp(text)]

# Preprocess data
processed_data = [(tokenize(comment), tokenize(reply)) for comment, reply in cleaned_data]

# Separate comments and replies
comments = [comment for comment, _ in processed_data]
replies = [reply for _, reply in processed_data]

# Create vocabulary
vocab = set()
for comment, reply in processed_data:
    vocab.update(comment)
    vocab.update(reply)

# Convert words to indices
word_to_idx = {word: idx for idx, word in enumerate(vocab)}

# Convert data to tensors
comments_tensor = torch.tensor([[word_to_idx[word] for word in comment] for comment, _ in processed_data])
replies_tensor = torch.tensor([[word_to_idx[word] for word in reply] for _, reply in processed_data])

# Define the Seq2Seq model
class Seq2SeqModel(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, output_dim):
        super(Seq2SeqModel, self).__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, src):
        embedded = self.embedding(src)
        output, hidden = self.rnn(embedded)
        return self.fc(hidden.squeeze(0))

# Set hyperparameters
input_dim = len(vocab)
output_dim = len(vocab)
emb_dim = 256
hidden_dim = 512
batch_size = 64

# Create model and optimizer
model = Seq2SeqModel(input_dim, emb_dim, hidden_dim, output_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Train the model (assuming you have labels/targets for the replies)
num_epochs = 10
for epoch in range(num_epochs):
    for comment, reply in zip(comments_tensor, replies_tensor):
        optimizer.zero_grad()
        output = model(comment.unsqueeze(0))
        loss = criterion(output, reply.unsqueeze(0))
        loss.backward()
        optimizer.step()

# Save the trained model
torch.save(model.state_dict(), 'seq2seq_model.pth')


/tmp/ipykernel_10174/4178662475.py:22: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
